In [ ]:
# Cell 1: Setup and Configuration

import requests
import json
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# --- CONFIGURATION ---
API_KEY = os.getenv("LINEAR_API_KEY")

# --- API Setup ---
API_URL = "https://api.linear.app/graphql"
HEADERS = {
    "Authorization": API_KEY, # No "Bearer" prefix
    "Content-Type": "application/json"
}

# Helper function to run GraphQL queries/mutations
def run_linear_query(query, variables=None):
    """A simple function to post a query to the Linear API."""
    payload = {'query': query}
    if variables:
        payload['variables'] = variables
    
    response = requests.post(API_URL, headers=HEADERS, json=payload)
    
    if response.status_code == 200:
        json_response = response.json()
        if "errors" in json_response:
            print(f"❌ GraphQL Error: {json_response['errors']}")
            return None
        return json_response
    else:
        raise Exception(f"Query failed to run with status code {response.status_code}: {response.text}")

if not API_KEY:
    print("❌ ERROR: LINEAR_API_KEY not found in environment variables.")
else:
    print("✅ Setup complete. API key loaded. Ready to proceed.")

In [ ]:
# Cell 2: Find Your Team ID

# ⚠️==================================================================
# ⚠️ Set your team's 3-letter key here.
# ⚠️ I'm assuming it's "LIN" from our previous conversations.
# ⚠️==================================================================
TEAM_KEY = "LIN"


print(f"🔎 Fetching ID for team '{TEAM_KEY}'...")
team_id_query = """
    query GetTeamId($teamKey: String!) {
      team(id: $teamKey) {
        id
        name
      }
    }
"""
team_id = None
if API_KEY:
    team_data = run_linear_query(team_id_query, {"teamKey": TEAM_KEY})
    
    if team_data and team_data.get('data', {}).get('team'):
        team_id = team_data['data']['team']['id']
        print(f"✅ Found Team: {team_data['data']['team']['name']} (ID: {team_id})")
    else:
        print(f"❌ Error: Could not find team with key '{TEAM_KEY}'. Please check the key in this cell.")
else:
     print("🚫 API Key not loaded. Please fix Cell 1.")

In [ ]:
# Cell 3: Fetch All Projects & Issues for That Team (Corrected)

# This query finds the team, gets all its projects,
# and then finds the outstanding issues for each project.
query = """
query($teamId: String!) {
  # 1. Find the team
  team(id: $teamId) {
    # 2. Get its projects (all of them)
    projects {
      nodes {
        id
        name
        state # <-- THIS IS THE CORRECTED LINE. It's a string, not an object.
        
        # 3. Get the outstanding issues for each project
        issues(
          filter: { state: { type: { nin: ["completed", "canceled"] } } }
        ) {
          nodes {
            id
            title
            identifier
            state { name } # This one is correct, issue states are objects.
          }
        }
      }
    }
  }
}
"""

if team_id:
    print(f"🔎 Fetching all projects for team '{TEAM_KEY}'...")
    data = run_linear_query(query, {"teamId": team_id})

    if data:
        print("✅ Query successful! Run the final cell to see the results.")
    else:
        print("❌ Query failed. Check the error message above.")
else:
    print("🚫 Team ID not found. Please fix Cell 2.")

In [ ]:
# Cell 4: Process and Display the Results (Corrected)

if 'data' in locals() and data:
    print("--- Report on Outstanding Issues ---")
    
    # Data is nested inside 'team' and 'projects'
    projects = data['data']['team']['projects']['nodes']
    found_issues_total = 0
    
    if not projects:
        print(f"\n🚫 No projects found at all in team '{TEAM_KEY}'.")
    
    # Loop through each project
    for project in projects:
        project_name = project['name']
        
        # --- THIS IS THE CORRECTED LINE ---
        # We just use the string directly.
        project_state = project['state'] if project['state'] else "Backlog"
        # ----------------------------------
        
        outstanding_issues = project['issues']['nodes']
        
        print(f"\n📁 Project: {project_name} (Status: {project_state})")
        
        # Print issues if they exist
        if outstanding_issues:
            for issue in outstanding_issues:
                print(f"  - [{issue['identifier']}] {issue['title']} (Status: {issue['state']['name']})")
                found_issues_total += 1
        else:
            # Otherwise, print that there are none
            print("  (No outstanding issues)")
            
    print("\n------------------------------------")
    
    if found_issues_total == 0 and projects:
        print("🎉 All projects are clear! No outstanding issues found.")
    elif found_issues_total > 0:
        print(f"Total outstanding issues found: {found_issues_total}")

else:
    print("🚫 No data to process. Please run Cell 3 first.")

### Send message via Discord

In [ ]:
# Cell 1: Setup and Configuration

import requests
import json
import os
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# --- CONFIGURATION ---
API_KEY = os.getenv("LINEAR_API_KEY")

# --- API Setup ---
# ⚠️ THIS IS THE CORRECTED LINE ⚠️
API_URL = "https://api.linear.app/graphql" # Was missing the "//"

HEADERS = {
    "Authorization": API_KEY, # No "Bearer" prefix
    "Content-Type": "application/json"
}

# Helper function to run GraphQL queries/mutations
def run_linear_query(query, variables=None):
    """A simple function to post a query to the Linear API."""
    payload = {'query': query}
    if variables:
        payload['variables'] = variables
    
    response = requests.post(API_URL, headers=HEADERS, json=payload)
    
    if response.status_code == 200:
        json_response = response.json()
        if "errors" in json_response:
            print(f"❌ GraphQL Error: {json_response['errors']}")
            return None
        return json_response
    else:
        raise Exception(f"Query failed to run with status code {response.status_code}: {response.text}")

if not API_KEY:
    print("❌ ERROR: LINEAR_API_KEY not found in environment variables.")
else:
    print("✅ Setup complete. API key loaded. Ready to proceed.")

In [ ]:
# Cell 2: Define and Run the Query

# 1. Get today's date AND time in ISO 8601 format
today_string = datetime.now().isoformat()

# 2. This query finds all projects that are currently active
#    (Start Date is in the past, Target Date is in the future)
#    and then finds their outstanding (not done) issues.
query = """
query($today: DateTimeOrDuration!) {
  projects(
    filter: {
      and: [
        { startDate: { lte: $today } },  # "has been started"
        { targetDate: { gte: $today } }  # "is not Done yet"
      ]
    }
  ) {
    nodes {
      id
      name
      url
      startDate
      targetDate
      teams {         # <-- MODIFIED: Changed to "teams"
        nodes {       # <-- ADDED
          name        # <-- ADDED
        }           # <-- ADDED
      }
      
      # For each project, get its outstanding issues
      issues(
        filter: { state: { type: { nin: ["completed", "canceled"] } } }
      ) {
        nodes {
          id
          title
          identifier
          url
          state { name }
        }
      }
    }
  }
}
"""

# 3. Pass today's full timestamp as a variable to the query
variables = {"today": today_string}

if API_KEY:
    print(f"🔎 Fetching active projects (current time is {today_string})...")
    data = run_linear_query(query, variables)

    if data:
        print("✅ Query successful! Run the next cell to build the bot message.")
    else:
        print("❌ Query failed. Check the error message above.")
else:
    print("🚫 API Key not loaded. Please fix Cell 1.")

In [ ]:
# Cell 3: Process Results and Format Notification

if 'data' in locals() and data:
    print("--- 🤖 Discord Bot Notification Report ---")
    
    projects_with_issues = []
    
    # Loop through all "In Progress" projects
    for project in data['data']['projects']['nodes']:
        outstanding_issues = project['issues']['nodes']
        
        # We only care about projects that HAVE outstanding issues
        if outstanding_issues:
            projects_with_issues.append(project)

    # --- Build the Message ---
    if not projects_with_issues:
        print("🎉 All active projects are clear! No outstanding issues to report.")
    
    else:
        # --- THIS IS THE MODIFIED SECTION ---
        
        # 1. Get the team name from the first project for the title
        team_name = "Unknown Team"
        if projects_with_issues[0]['teams']['nodes']:
            team_name = projects_with_issues[0]['teams']['nodes'][0]['name']

        # 2. Build the message list
        discord_message = []
        discord_message.append(f"🔔 **Team {team_name} Alert!**") # <-- Dynamic title
        discord_message.append("The following active projects have outstanding issues:")
        # ----------------------------------
        
        for project in projects_with_issues:
            # Get the team name (can be different if projects span teams)
            proj_team_name = "Unknown Team"
            if project['teams']['nodes']:
                proj_team_name = project['teams']['nodes'][0]['name']
            
            target_date = project['targetDate'].split('T')[0] if project['targetDate'] else "Not set"
            
            # Add the Project title and its dates
            discord_message.append(
                f"\n📁 **Project: {project['name']}** (Team: {proj_team_name} | Due on: {target_date})"
            )
            
            discord_message.append(f"   ( <{project['url']}> )")
            
            # Add each issue
            for issue in project['issues']['nodes']:
                discord_message.append(
                    f"  - **{issue['identifier']}**: {issue['title']} (Status: *{issue['state']['name']}*)"
                )
        
        print("\n".join(discord_message))

else:
    print("🚫 No data to process. Please run Cell 2 first.")